# Jagat Literasi

In [111]:
import pandas as pd
import re
import os

In [112]:
df = pd.read_csv("../data/raw/jagat_literasi.csv")
df.head()

,url,category,subcategory,tanggal_publish,judul,tags,konten
0,http://regional.kompas.com/read/2025/11/01/202...,Jagat Literasi,Regional,"1 November 2025, 20:21 WIB","Buku Datang ke Tobelo, Semangat Literasi Siswa...","donasi buku, halmahera utara, sekolah swasta, ...",KOMPAS.com – Sekolah swasta juga memiliki pera...
1,http://regional.kompas.com/read/2025/11/01/193...,Jagat Literasi,Regional,"1 November 2025, 19:33 WIB","Dapat 450 Buku Donasi, Impian SD Alkhairaat To...","kompascom, donasi buku, Jagat Literasi, Nursia...","HALMAHERA UTARA, KOMPAS.com – Bel pagi berbuny..."
2,http://megapolitan.kompas.com/read/2025/10/17/...,Jagat Literasi,Megapolitan,"17 Oktober 2025, 06:00 WIB","Dorong Akses Pendidikan, Komunitas Generasi Je...","donasi buku, akses pendidikan, STEM, sekolah n...",KOMPAS.com - Di tengah pesatnya perkembangan k...
3,http://surabaya.kompas.com/read/2025/09/30/110...,Jagat Literasi,Surabaya,"30 September 2025, 11:07 WIB","Kini Tak Berdaya, Pedagang Buku Bekas Pasar Bl...","toko buku bekas, kondisi pasar, berita jatim, ...","SURABAYA, KOMPAS.com - Salah satu pasar buku b..."
4,http://surabaya.kompas.com/read/2025/09/26/101...,Jagat Literasi,Surabaya,"26 September 2025, 10:13 WIB","Toko Buku Togamas Margorejo Surabaya, Dulu Pen...","Surabaya, Jagat Literasi, Togamas, Toko buku t...","SURABAYA, KOMPAS.com - Bagi masyarakat Surabay..."


In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   url              210 non-null    object
 1   category         210 non-null    object
 2   subcategory      210 non-null    object
 3   tanggal_publish  209 non-null    object
 4   judul            210 non-null    object
 5   tags             208 non-null    object
 6   konten           210 non-null    object
dtypes: object(7)
memory usage: 11.6+ KB


In [114]:
df.rename(columns={
    'judul': 'judul_berita',
    'konten': 'konten_berita'
}, inplace=True)

In [115]:
df.columns.tolist()

['url',
 'category',
 'subcategory',
 'tanggal_publish',
 'judul_berita',
 'tags',
 'konten_berita']

## Null Handling

In [116]:
df['konten_berita'].isna().sum()

0

## Remove Duplicate

In [117]:
df.duplicated().sum()

21

In [118]:
df = df.drop_duplicates()

In [119]:
df.duplicated().sum()

0

## Perbaiki Penulisan

In [120]:
def clean_text(text):
    # mengganti newline dengan spasi
    text = text.replace("\n", " ")
    # hapus spasi berlebih
    text = re.sub(r'\s+', ' ', text).strip()
    # spasi setelah tanda baca jika belum ada
    text = re.sub(r'([.,;:!?])(?!\s)', r'\1 ', text)
    # domain
    #text = re.sub(r'\.(?!\s|[a-zA-Z])', r'. ', text)
    # hapus spasi berlebih setelah penambahan
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [121]:
df['konten_berita'] = df['konten_berita'].apply(clean_text)
print(df[['konten_berita']].head())

                                       konten_berita
0  KOMPAS. com – Sekolah swasta juga memiliki per...
1  HALMAHERA UTARA, KOMPAS. com – Bel pagi berbun...
2  KOMPAS. com - Di tengah pesatnya perkembangan ...
3  SURABAYA, KOMPAS. com - Salah satu pasar buku ...
4  SURABAYA, KOMPAS. com - Bagi masyarakat Suraba...


In [122]:
df['judul_berita'] = df['judul_berita'].apply(clean_text)
print(df[['judul_berita']].head())

                                        judul_berita
0  Buku Datang ke Tobelo, Semangat Literasi Siswa...
1  Dapat 450 Buku Donasi, Impian SD Alkhairaat To...
2  Dorong Akses Pendidikan, Komunitas Generasi Je...
3  Kini Tak Berdaya, Pedagang Buku Bekas Pasar Bl...
4  Toko Buku Togamas Margorejo Surabaya, Dulu Pen...


## Hapus Atribut Teks Tidak Penting

In [123]:
import re

def clean_kompas_text(text: str) -> str:
    # ubah kompas. com menjadi kompas.com
    text = re.sub(r'(?i)kompas\s*\.\s*com', 'kompas.com', text)

    # hapus prefiks lokasi, KOMPAS.com – ...
    text = re.sub(
        r'^[A-Z\s]+,\s*kompas\.com\s*[–\-—]\s*',
        '',
        text,
        flags=re.IGNORECASE
    )

    # hapus kompas doc
    text = re.sub(r'(?is)kompas\.com/\s*dok\.[^.]*\.', '', text)

    # hapus kompas.com
    text = re.sub(r'(?i)(?:[A-Za-z]*\s*[-–\\\/]*)?kompas\.com[\\/A-Za-z0-9\-]*', '', text)

    # gabungan
    text = re.sub(r'^[A-Za-z\s]*?[a-z](?=[A-Z])', '', text)

    # hapus baca, baca juga, dan kombinasinya
    text = re.sub(
        r'(?i)\(?baca(?:\s+(?:juga|selengkapnya))?\s*[:：]\s*[^.!?\n]*(?=[.!?\n]|$)',
        '',
        text
    )

    # mention
    text = re.sub(r'\(@[^)]+\)', '', text)
    text = re.sub(r'@\w+', '', text)

    # kredit kompas.com
    text = re.sub(r'(?i)\bkompas\.com/[\w\s.]+?[a-z](?=\s*[A-Z]{2,}|[A-Z][a-z]*[A-Z])', '', text)

    # hapus .com yang mungkin tersisa
    text = re.sub(r'\b\w*\.com\b', '', text, flags=re.IGNORECASE)

    # tanda baca dan spasi
    text = re.sub(r'\s+([.,!?;:])', r'\1', text)
    text = re.sub(r'([.,!?;:])([^\s"”’])', r'\1 \2', text)
    text = re.sub(r'(?<=\w)\(', r' (', text)
    text = re.sub(r'\(\s+', '(', text)
    text = re.sub(r'\s+\)', ')', text)
    text = re.sub(r'\)(?=[A-Za-z0-9])', r') ', text)
    text = re.sub(r'\s+(")', r'\1', text)
    text = re.sub(r'\.\s*"', '. "', text)

    # tanda baca berulang
    text = re.sub(r'([.,!?;:])(\s*\1)+', r'\1', text)
    text = re.sub(r'([.,!?;:])\s*[.,!?;:]+', r'\1', text)
    text = re.sub(r'"\s*"', '"', text)
    text = re.sub(r'""+', '"', text)

    # non huruf awal karakter
    text = re.sub(r'^[^A-Za-z0-9]+', '', text)

    # spasi ganda
    text = re.sub(r'\s{2,}', ' ', text)

    # trim spasi
    text = text.strip()

    return text


In [124]:
print(df[['konten_berita']].head())

                                       konten_berita
0  KOMPAS. com – Sekolah swasta juga memiliki per...
1  HALMAHERA UTARA, KOMPAS. com – Bel pagi berbun...
2  KOMPAS. com - Di tengah pesatnya perkembangan ...
3  SURABAYA, KOMPAS. com - Salah satu pasar buku ...
4  SURABAYA, KOMPAS. com - Bagi masyarakat Suraba...


In [125]:
df['konten_berita'] = df['konten_berita'].apply(clean_kompas_text)
print(df[['konten_berita']].head())

                                       konten_berita
0  Sekolah swasta juga memiliki peran strategis d...
1  Bel pagi berbunyi, pertanda waktu apel masuk s...
2  Di tengah pesatnya perkembangan kota besar, ma...
3  Salah satu pasar buku bekas paling ikonik di S...
4  Bagi masyarakat Surabaya, Jawa Timur, sudah ta...


In [126]:
df['judul_berita'] = df['judul_berita'].apply(clean_kompas_text)
print(df[['judul_berita']].head())

                                        judul_berita
0  Buku Datang ke Tobelo, Semangat Literasi Siswa...
1  Dapat 450 Buku Donasi, Impian SD Alkhairaat To...
2  Dorong Akses Pendidikan, Komunitas Generasi Je...
3  Kini Tak Berdaya, Pedagang Buku Bekas Pasar Bl...
4  Toko Buku Togamas Margorejo Surabaya, Dulu Pen...


In [127]:
df["judul_berita"] = df["judul_berita"].apply(
    lambda x: x.strip() + "." if not re.search(r'[.!?…]$', str(x).strip()) else x.strip()
)

## Save Hasil Preprocessing

In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 189 entries, 0 to 209
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   url              189 non-null    object
 1   category         189 non-null    object
 2   subcategory      189 non-null    object
 3   tanggal_publish  188 non-null    object
 4   judul_berita     189 non-null    object
 5   tags             187 non-null    object
 6   konten_berita    189 non-null    object
dtypes: object(7)
memory usage: 11.8+ KB


In [129]:
df.duplicated().sum()

0

In [ ]:
df.to_csv("../data/preprocessing/prepro_jagat_literasi.csv", index=False)

# Cahaya

## Cahaya Aktual

In [131]:
df = pd.read_csv("../data/raw/cahaya_aktual.csv")
df.head()

,url,category,subcategory,tanggal_publish,judul,tags,konten
0,https://cahaya.kompas.com/aktual/25K1217584099...,Cahaya,Aktual,"Kompas.com - 12/11/2025, 17:58 WIB",Menag Tegaskan Tak Toleransi Kekerasan dan Pel...,"Kemenag, Menag, kekerasan di pesantren, pelece...",KOMPAS.com -Menteri Agama Nasaruddin Umar mene...
1,https://cahaya.kompas.com/aktual/25K1216380259...,Cahaya,Aktual,"Kompas.com - 12/11/2025, 16:38 WIB",Wali Hakim dalam Akad Nikah: Dasar Hukum dan K...,"nikah, akad nikah, wali hakim, wali nasab","KOMPAS.com -Dalam prosesi akad nikah, kehadira..."
2,https://cahaya.kompas.com/aktual/25K1215315829...,Cahaya,Aktual,"Kompas.com - 12/11/2025, 15:31 WIB",BPKH Gandeng Buzz ARVR Wujudkan Inovasi Haji D...,"BPKH, haji 2026, haji digital, Buzz ARVR",KOMPAS.com -Badan Pengelola Keuangan Haji (BPK...
3,https://cahaya.kompas.com/aktual/25K1210472569...,Cahaya,Aktual,"Kompas.com - 12/11/2025, 10:47 WIB",Ucapan Menyentuh Hati untuk Peringatan Hari Ay...,"Hari Ayah Nasional, 12 November, peringatan ha...",KOMPAS.com - Hari Ayah Nasional diperingati se...
4,https://cahaya.kompas.com/aktual/25K1209035389...,Cahaya,Aktual,"Kompas.com - 12/11/2025, 09:03 WIB","Jelang Hari Guru, 101.786 Guru Madrasah dan Gu...","Guru agama, Menag, guru madrasah, Hari Guru, P...",KOMPAS.com -Menjelang peringatan Hari Guru Nas...


In [132]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   url              200 non-null    object
 1   category         200 non-null    object
 2   subcategory      200 non-null    object
 3   tanggal_publish  200 non-null    object
 4   judul            200 non-null    object
 5   tags             200 non-null    object
 6   konten           200 non-null    object
dtypes: object(7)
memory usage: 11.1+ KB


In [133]:
df.rename(columns={
    'judul': 'judul_berita',
    'konten': 'konten_berita'
}, inplace=True)

In [134]:
df.columns.tolist()

['url',
 'category',
 'subcategory',
 'tanggal_publish',
 'judul_berita',
 'tags',
 'konten_berita']

### Null Handling

In [135]:
df['konten_berita'].isna().sum()

0

### Remove Duplicate

In [136]:
df.duplicated().sum()

0

### Perbaiki Penulisan

In [137]:
import re

def remove_before_first_number(text: str) -> str:
    text = re.sub(r'^[^\d]*', '', text).strip()
    text = re.sub(r':\s+', ':', text)
    
    return text


In [138]:
print(df[['tanggal_publish']].head())

                      tanggal_publish
0  Kompas.com - 12/11/2025, 17:58 WIB
1  Kompas.com - 12/11/2025, 16:38 WIB
2  Kompas.com - 12/11/2025, 15:31 WIB
3  Kompas.com - 12/11/2025, 10:47 WIB
4  Kompas.com - 12/11/2025, 09:03 WIB


In [139]:
df['tanggal_publish'] = df['tanggal_publish'].apply(remove_before_first_number)
print(df[['tanggal_publish']].head())

         tanggal_publish
0  12/11/2025, 17:58 WIB
1  12/11/2025, 16:38 WIB
2  12/11/2025, 15:31 WIB
3  12/11/2025, 10:47 WIB
4  12/11/2025, 09:03 WIB


In [140]:
def clean_text(text):
    # mengganti newline dengan spasi
    text = text.replace("\n", " ")
    # hapus spasi berlebih
    text = re.sub(r'\s+', ' ', text).strip()
    # spasi setelah tanda baca jika belum ada
    text = re.sub(r'([.,;:!?])(?!\s)', r'\1 ', text)
    # domain
    #text = re.sub(r'\.(?!\s|[a-zA-Z])', r'. ', text)
    # hapus spasi berlebih setelah penambahan
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [141]:
df['konten_berita'] = df['konten_berita'].apply(clean_text)
print(df[['konten_berita']].head())

                                       konten_berita
0  KOMPAS. com -Menteri Agama Nasaruddin Umar men...
1  KOMPAS. com -Dalam prosesi akad nikah, kehadir...
2  KOMPAS. com -Badan Pengelola Keuangan Haji (BP...
3  KOMPAS. com - Hari Ayah Nasional diperingati s...
4  KOMPAS. com -Menjelang peringatan Hari Guru Na...


In [142]:
df['judul_berita'] = df['judul_berita'].apply(clean_text)
print(df[['judul_berita']].head())

                                        judul_berita
0  Menag Tegaskan Tak Toleransi Kekerasan dan Pel...
1  Wali Hakim dalam Akad Nikah: Dasar Hukum dan K...
2  BPKH Gandeng Buzz ARVR Wujudkan Inovasi Haji D...
3  Ucapan Menyentuh Hati untuk Peringatan Hari Ay...
4  Jelang Hari Guru, 101. 786 Guru Madrasah dan G...


### Hapus Atribut Teks Tidak Penting

In [143]:
def clean_kompas_text(text: str) -> str:
    # ubah kompas. com menjadi kompas.com
    text = re.sub(r'(?i)kompas\s*\.\s*com', 'kompas.com', text)

    # hapus prefiks lokasi, KOMPAS.com – ...
    text = re.sub(
        r'^[A-Z\s]+,\s*kompas\.com\s*[–\-—]\s*',
        '',
        text,
        flags=re.IGNORECASE
    )

    # hapus kompas doc
    text = re.sub(r'(?is)kompas\.com/\s*dok\.[^.]*\.', '', text)

    # hapus kompas.com
    text = re.sub(r'(?i)(?:[A-Za-z]*\s*[-–\\\/]*)?kompas\.com[\\/A-Za-z0-9\-]*', '', text)

    # gabungan
    text = re.sub(r'^[A-Za-z\s]*?[a-z](?=[A-Z])', '', text)

    # hapus baca, baca juga, dan kombinasinya
    text = re.sub(
        r'(?i)\(?baca(?:\s+(?:juga|selengkapnya))?\s*[:：]\s*[^.!?\n]*(?=[.!?\n]|$)',
        '',
        text
    )

    # mention
    text = re.sub(r'\(@[^)]+\)', '', text)
    text = re.sub(r'@\w+', '', text)

    # kredit kompas.com
    text = re.sub(r'(?i)\bkompas\.com/[\w\s.]+?[a-z](?=\s*[A-Z]{2,}|[A-Z][a-z]*[A-Z])', '', text)

    # hapus .com yang mungkin tersisa
    text = re.sub(r'\b\w*\.com\b', '', text, flags=re.IGNORECASE)

    # tanda baca dan spasi
    text = re.sub(r'\s+([.,!?;:])', r'\1', text)
    text = re.sub(r'([.,!?;:])([^\s"”’])', r'\1 \2', text)
    text = re.sub(r'(?<=\w)\(', r' (', text)
    text = re.sub(r'\(\s+', '(', text)
    text = re.sub(r'\s+\)', ')', text)
    text = re.sub(r'\)(?=[A-Za-z0-9])', r') ', text)
    text = re.sub(r'\s+(")', r'\1', text)
    text = re.sub(r'\.\s*"', '. "', text)

    # tanda baca berulang
    text = re.sub(r'([.,!?;:])(\s*\1)+', r'\1', text)
    text = re.sub(r'([.,!?;:])\s*[.,!?;:]+', r'\1', text)
    text = re.sub(r'"\s*"', '"', text)
    text = re.sub(r'""+', '"', text)

    # non huruf awal karakter
    text = re.sub(r'^[^A-Za-z0-9]+', '', text)

    # spasi ganda
    text = re.sub(r'\s{2,}', ' ', text)

    # trim spasi
    text = text.strip()

    return text


In [144]:
print(df[['konten_berita']].head())

                                       konten_berita
0  KOMPAS. com -Menteri Agama Nasaruddin Umar men...
1  KOMPAS. com -Dalam prosesi akad nikah, kehadir...
2  KOMPAS. com -Badan Pengelola Keuangan Haji (BP...
3  KOMPAS. com - Hari Ayah Nasional diperingati s...
4  KOMPAS. com -Menjelang peringatan Hari Guru Na...


In [145]:
df['konten_berita'] = df['konten_berita'].apply(clean_kompas_text)
print(df[['konten_berita']].head())

                                       konten_berita
0  Menteri Agama Nasaruddin Umar menegaskan bahwa...
1  Dalam prosesi akad nikah, kehadiran seorang wa...
2  Badan Pengelola Keuangan Haji (BPKH) menandata...
3  Hari Ayah Nasional diperingati setiap tanggal ...
4  Menjelang peringatan Hari Guru Nasional 25 Nov...


In [146]:
print(df[['judul_berita']].head())

                                        judul_berita
0  Menag Tegaskan Tak Toleransi Kekerasan dan Pel...
1  Wali Hakim dalam Akad Nikah: Dasar Hukum dan K...
2  BPKH Gandeng Buzz ARVR Wujudkan Inovasi Haji D...
3  Ucapan Menyentuh Hati untuk Peringatan Hari Ay...
4  Jelang Hari Guru, 101. 786 Guru Madrasah dan G...


In [147]:
df['judul_berita'] = df['judul_berita'].apply(clean_kompas_text)
print(df[['judul_berita']].head())

                                        judul_berita
0  Menag Tegaskan Tak Toleransi Kekerasan dan Pel...
1  Wali Hakim dalam Akad Nikah: Dasar Hukum dan K...
2  BPKH Gandeng Buzz ARVR Wujudkan Inovasi Haji D...
3  Ucapan Menyentuh Hati untuk Peringatan Hari Ay...
4  Jelang Hari Guru, 101. 786 Guru Madrasah dan G...


In [148]:
df["judul_berita"] = df["judul_berita"].apply(
    lambda x: x.strip() + "." if not re.search(r'[.!?…]$', str(x).strip()) else x.strip()
)

### Save Hasil Preprocessing

In [149]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   url              200 non-null    object
 1   category         200 non-null    object
 2   subcategory      200 non-null    object
 3   tanggal_publish  200 non-null    object
 4   judul_berita     200 non-null    object
 5   tags             200 non-null    object
 6   konten_berita    200 non-null    object
dtypes: object(7)
memory usage: 11.1+ KB


In [150]:
df.duplicated().sum()

0

In [151]:
df.to_csv("../data/preprocessing/prepro_cahaya_aktual.csv", index=False)

## Cahaya Doa dan Niat

In [152]:
df = pd.read_csv("../data/raw/cahaya_doa_dan_niat.csv")
df.head()

,url,category,subcategory,tanggal_publish,judul,tags,konten
0,https://cahaya.kompas.com/doa-dan-niat/25K1218...,Cahaya,Doa dan Niat,"Kompas.com - 12/11/2025, 18:20 WIB","Surat Al Zalzalah: Bacaan, Arti, Asbabun Nuzul...","surat al zalzalah, surat ke-99 dalam Al Quran,...",KOMPAS.com - Surat Al Zalzalah adalah surat ke...
1,https://cahaya.kompas.com/doa-dan-niat/25K1216...,Cahaya,Doa dan Niat,"Kompas.com - 12/11/2025, 16:32 WIB","Makna Fitnah dalam Islam, Bukan Sekedar Tuduha...","fitnah, pengertian fitnah, macam-macam fitnah",KOMPAS.com - Fitnah sering diartikan dengan tu...
2,https://cahaya.kompas.com/doa-dan-niat/25K1215...,Cahaya,Doa dan Niat,"Kompas.com - 12/11/2025, 15:34 WIB",Panduan Lengkap Shalat Taubat Disertai dengan ...,"shalat taubat, mandi taubat, doa shalat taubat...",KOMPAS.com - Taubat adalah salah satu cara men...
3,https://cahaya.kompas.com/doa-dan-niat/25K1213...,Cahaya,Doa dan Niat,"Kompas.com - 12/11/2025, 13:28 WIB","Keutamaan Menjenguk Orang Sakit, Didoakan 70 R...","menjenguk orang sakit, orang sakit, perintah m...","KOMPAS.com - Dalam kehidupan bermasyarakat, se..."
4,https://cahaya.kompas.com/doa-dan-niat/25K1210...,Cahaya,Doa dan Niat,"Kompas.com - 12/11/2025, 10:13 WIB",Doa untuk Ayah di Hari Ayah Nasional Tanggal 1...,"Hari Ayah Nasional, Hari Ayah 12 November, tan...",KOMPAS.com - Hari Ayah Nasional diperingati se...


In [153]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   url              200 non-null    object
 1   category         200 non-null    object
 2   subcategory      200 non-null    object
 3   tanggal_publish  200 non-null    object
 4   judul            200 non-null    object
 5   tags             200 non-null    object
 6   konten           200 non-null    object
dtypes: object(7)
memory usage: 11.1+ KB


In [154]:
df.rename(columns={
    'judul': 'judul_berita',
    'konten': 'konten_berita'
}, inplace=True)

In [155]:
df.columns.tolist()

['url',
 'category',
 'subcategory',
 'tanggal_publish',
 'judul_berita',
 'tags',
 'konten_berita']

### Null Handling

In [156]:
df['konten_berita'].isna().sum()

0

### Remove Duplicate

In [157]:
df.duplicated().sum()

0

### Perbaiki Penulisan

In [158]:
import re

def remove_before_first_number(text: str) -> str:
    text = re.sub(r'^[^\d]*', '', text).strip()
    text = re.sub(r':\s+', ':', text)
    
    return text


In [159]:
print(df[['tanggal_publish']].head())

                      tanggal_publish
0  Kompas.com - 12/11/2025, 18:20 WIB
1  Kompas.com - 12/11/2025, 16:32 WIB
2  Kompas.com - 12/11/2025, 15:34 WIB
3  Kompas.com - 12/11/2025, 13:28 WIB
4  Kompas.com - 12/11/2025, 10:13 WIB


In [160]:
df['tanggal_publish'] = df['tanggal_publish'].apply(remove_before_first_number)
print(df[['tanggal_publish']].head())

         tanggal_publish
0  12/11/2025, 18:20 WIB
1  12/11/2025, 16:32 WIB
2  12/11/2025, 15:34 WIB
3  12/11/2025, 13:28 WIB
4  12/11/2025, 10:13 WIB


In [161]:
def clean_text(text):
    # mengganti newline dengan spasi
    text = text.replace("\n", " ")
    # hapus spasi berlebih
    text = re.sub(r'\s+', ' ', text).strip()
    # spasi setelah tanda baca jika belum ada
    text = re.sub(r'([.,;:!?])(?!\s)', r'\1 ', text)
    # domain
    #text = re.sub(r'\.(?!\s|[a-zA-Z])', r'. ', text)
    # hapus spasi berlebih setelah penambahan
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [162]:
df['konten_berita'] = df['konten_berita'].apply(clean_text)
print(df[['konten_berita']].head())

                                       konten_berita
0  KOMPAS. com - Surat Al Zalzalah adalah surat k...
1  KOMPAS. com - Fitnah sering diartikan dengan t...
2  KOMPAS. com - Taubat adalah salah satu cara me...
3  KOMPAS. com - Dalam kehidupan bermasyarakat, s...
4  KOMPAS. com - Hari Ayah Nasional diperingati s...


In [163]:
df['judul_berita'] = df['judul_berita'].apply(clean_text)
print(df[['judul_berita']].head())

                                        judul_berita
0  Surat Al Zalzalah: Bacaan, Arti, Asbabun Nuzul...
1  Makna Fitnah dalam Islam, Bukan Sekedar Tuduha...
2  Panduan Lengkap Shalat Taubat Disertai dengan ...
3  Keutamaan Menjenguk Orang Sakit, Didoakan 70 R...
4  Doa untuk Ayah di Hari Ayah Nasional Tanggal 1...


### Hapus Atribut Teks Tidak Penting

In [164]:
def clean_kompas_text(text: str) -> str:
    # ubah kompas. com menjadi kompas.com
    text = re.sub(r'(?i)kompas\s*\.\s*com', 'kompas.com', text)

    # hapus prefiks lokasi, KOMPAS.com – ...
    text = re.sub(
        r'^[A-Z\s]+,\s*kompas\.com\s*[–\-—]\s*',
        '',
        text,
        flags=re.IGNORECASE
    )

    # hapus kompas doc
    text = re.sub(r'(?is)kompas\.com/\s*dok\.[^.]*\.', '', text)

    # hapus kompas.com
    text = re.sub(r'(?i)(?:[A-Za-z]*\s*[-–\\\/]*)?kompas\.com[\\/A-Za-z0-9\-]*', '', text)

    # gabungan
    text = re.sub(r'^[A-Za-z\s]*?[a-z](?=[A-Z])', '', text)

    # hapus baca, baca juga, dan kombinasinya
    text = re.sub(
        r'(?i)\(?baca(?:\s+(?:juga|selengkapnya))?\s*[:：]\s*[^.!?\n]*(?=[.!?\n]|$)',
        '',
        text
    )

    # mention
    text = re.sub(r'\(@[^)]+\)', '', text)
    text = re.sub(r'@\w+', '', text)

    # kredit kompas.com
    text = re.sub(r'(?i)\bkompas\.com/[\w\s.]+?[a-z](?=\s*[A-Z]{2,}|[A-Z][a-z]*[A-Z])', '', text)

    # hapus .com yang mungkin tersisa
    text = re.sub(r'\b\w*\.com\b', '', text, flags=re.IGNORECASE)

    # tanda baca dan spasi
    text = re.sub(r'\s+([.,!?;:])', r'\1', text)
    text = re.sub(r'([.,!?;:])([^\s"”’])', r'\1 \2', text)
    text = re.sub(r'(?<=\w)\(', r' (', text)
    text = re.sub(r'\(\s+', '(', text)
    text = re.sub(r'\s+\)', ')', text)
    text = re.sub(r'\)(?=[A-Za-z0-9])', r') ', text)
    text = re.sub(r'\s+(")', r'\1', text)
    text = re.sub(r'\.\s*"', '. "', text)

    # tanda baca berulang
    text = re.sub(r'([.,!?;:])(\s*\1)+', r'\1', text)
    text = re.sub(r'([.,!?;:])\s*[.,!?;:]+', r'\1', text)
    text = re.sub(r'"\s*"', '"', text)
    text = re.sub(r'""+', '"', text)

    # non huruf awal karakter
    text = re.sub(r'^[^A-Za-z0-9]+', '', text)

    # spasi ganda
    text = re.sub(r'\s{2,}', ' ', text)

    # trim spasi
    text = text.strip()

    return text

In [165]:
print(df[['konten_berita']].head())

                                       konten_berita
0  KOMPAS. com - Surat Al Zalzalah adalah surat k...
1  KOMPAS. com - Fitnah sering diartikan dengan t...
2  KOMPAS. com - Taubat adalah salah satu cara me...
3  KOMPAS. com - Dalam kehidupan bermasyarakat, s...
4  KOMPAS. com - Hari Ayah Nasional diperingati s...


In [166]:
df['konten_berita'] = df['konten_berita'].apply(clean_kompas_text)
print(df[['konten_berita']].head())

                                       konten_berita
0  Surat Al Zalzalah adalah surat ke-99 dalam Al ...
1  Fitnah sering diartikan dengan tuduhan atau be...
2  Taubat adalah salah satu cara mensucikan diri ...
3  Dalam kehidupan bermasyarakat, seorang muslim ...
4  Hari Ayah Nasional diperingati setiap tanggal ...


In [167]:
print(df[['judul_berita']].head())

                                        judul_berita
0  Surat Al Zalzalah: Bacaan, Arti, Asbabun Nuzul...
1  Makna Fitnah dalam Islam, Bukan Sekedar Tuduha...
2  Panduan Lengkap Shalat Taubat Disertai dengan ...
3  Keutamaan Menjenguk Orang Sakit, Didoakan 70 R...
4  Doa untuk Ayah di Hari Ayah Nasional Tanggal 1...


In [168]:
df['judul_berita'] = df['judul_berita'].apply(clean_kompas_text)
print(df[['judul_berita']].head())

                                        judul_berita
0  Surat Al Zalzalah: Bacaan, Arti, Asbabun Nuzul...
1  Makna Fitnah dalam Islam, Bukan Sekedar Tuduha...
2  Panduan Lengkap Shalat Taubat Disertai dengan ...
3  Keutamaan Menjenguk Orang Sakit, Didoakan 70 R...
4  Doa untuk Ayah di Hari Ayah Nasional Tanggal 1...


In [169]:
df["judul_berita"] = df["judul_berita"].apply(
    lambda x: x.strip() + "." if not re.search(r'[.!?…]$', str(x).strip()) else x.strip()
)

### Save Hasil Preprocessing

In [170]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   url              200 non-null    object
 1   category         200 non-null    object
 2   subcategory      200 non-null    object
 3   tanggal_publish  200 non-null    object
 4   judul_berita     200 non-null    object
 5   tags             200 non-null    object
 6   konten_berita    200 non-null    object
dtypes: object(7)
memory usage: 11.1+ KB


In [171]:
df.duplicated().sum()

0

In [172]:
df.to_csv("../data/preprocessing/prepro_cahaya_doa_dan_niat.csv", index=False)

# JEO

In [173]:
df = pd.read_csv("../data/raw/jeo.csv")
df.head()

,category,subcategory,title,url,summary,date_published,content
0,JEO,Tokoh,Pidato Lengkap Presiden Prabowo dalam Sidang K...,https://jeo.kompas.com/pidato-lengkap-presiden...,"Tepat pada satu tahun pemerintahannya, Preside...","Selasa, 21 Oktober 2025 | 16:56 WIB","TEPATpada satu tahun pemerintahannya, Presiden..."
1,JEO,Peristiwa,Mimpi yang Tak Padam di Tapal Batas: Cerita An...,https://jeo.kompas.com/mimpi-yang-tak-padam-di...,Sebuah kisah tentang anak-anak yang sekolahnya...,"Kamis, 11 September 2025 | 14:08 WIB",Sebuah kisah tentang anak-anak yang sekolahnya...
2,JEO,Peristiwa,Antara Asa dan Cicilan: Kisah dan Strategi Par...,https://jeo.kompas.com/antara-asa-dan-cicilan-...,Di balik kenyamanan memiliki tempat tinggal se...,"Jumat, 18 Juli 2025 | 13:02 WIB",Di balik kenyamanan memiliki tempat tinggal se...
3,JEO,Peristiwa,"Yang Muda, yang Tua, yang Terlupa: Realitas Pa...",https://jeo.kompas.com/yang-muda-yang-tua-yang...,Di tengah lautan map cokelat dan antrean yang ...,"Selasa, 17 Juni 2025 | 07:06 WIB",Di tengah lautan map cokelat dan antrean yang ...
4,JEO,Peristiwa,"Mengupas Masa Depan Nissan di Indonesia, mulai...",https://jeo.kompas.com/mengupas-masa-depan-nis...,Seiring dengan peningkatan pemahaman dan kebut...,"Rabu, 14 Mei 2025 | 12:57 WIB",Seiring dengan peningkatan pemahaman dan kebut...


In [174]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 510 entries, 0 to 509
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   category        510 non-null    object
 1   subcategory     510 non-null    object
 2   title           510 non-null    object
 3   url             510 non-null    object
 4   summary         510 non-null    object
 5   date_published  499 non-null    object
 6   content         500 non-null    object
dtypes: object(7)
memory usage: 28.0+ KB


In [175]:
df.rename(columns={
    'title': 'judul_berita',
    'content': 'konten_berita',
    'date_published': 'tanggal_publish'
}, inplace=True)

In [176]:
df.columns.tolist()

['category',
 'subcategory',
 'judul_berita',
 'url',
 'summary',
 'tanggal_publish',
 'konten_berita']

## Null Handling

In [177]:
df['konten_berita'].isna().sum()

10

In [178]:
df = df.dropna()

In [179]:
df['konten_berita'].isna().sum()

0

## Remove Duplicate

In [180]:
df.duplicated().sum()

0

## Perbaiki Penulisan

In [181]:
import re

def clean_date_text(text: str) -> str:
    text = re.sub(r'^[^\d]*', '', text).strip()

    text = re.sub(r':\s+', ':', text)

    text = re.sub(r'\s*\|\s*', ', ', text)

    text = re.sub(r'\s{2,}', ' ', text).strip()

    return text


In [182]:
print(df[['tanggal_publish']].head())

                        tanggal_publish
0   Selasa, 21 Oktober 2025 | 16:56 WIB
1  Kamis, 11 September 2025 | 14:08 WIB
2       Jumat, 18 Juli 2025 | 13:02 WIB
3      Selasa, 17 Juni 2025 | 07:06 WIB
4         Rabu, 14 Mei 2025 | 12:57 WIB


In [183]:
df['tanggal_publish'] = df['tanggal_publish'].apply(clean_date_text)
print(df[['tanggal_publish']].head())

                tanggal_publish
0    21 Oktober 2025, 16:56 WIB
1  11 September 2025, 14:08 WIB
2       18 Juli 2025, 13:02 WIB
3       17 Juni 2025, 07:06 WIB
4        14 Mei 2025, 12:57 WIB


In [185]:
def replace_first_sentence(row):
    summary = str(row['summary']).strip()
    konten = str(row['konten_berita']).strip()

    match = re.match(r'(.+?[.!?])(\s|$)(.*)', konten, flags=re.DOTALL)
    if match:
        return summary + ' ' + match.group(3).strip()
    else:
        return summary


In [186]:
print(df[['konten_berita']].head())

                                       konten_berita
0  TEPATpada satu tahun pemerintahannya, Presiden...
1  Sebuah kisah tentang anak-anak yang sekolahnya...
2  Di balik kenyamanan memiliki tempat tinggal se...
3  Di tengah lautan map cokelat dan antrean yang ...
4  Seiring dengan peningkatan pemahaman dan kebut...


In [188]:
df['konten_berita'] = df.apply(replace_first_sentence, axis=1)
print(df[['konten_berita']].head())

                                       konten_berita
0  Tepat pada satu tahun pemerintahannya, Preside...
1  Sebuah kisah tentang anak-anak yang sekolahnya...
2  Di balik kenyamanan memiliki tempat tinggal se...
3  Di tengah lautan map cokelat dan antrean yang ...
4  Seiring dengan peningkatan pemahaman dan kebut...


In [189]:
def clean_text(text):
    # mengganti newline dengan spasi
    text = text.replace("\n", " ")
    # hapus spasi berlebih
    text = re.sub(r'\s+', ' ', text).strip()
    # spasi setelah tanda baca jika belum ada
    text = re.sub(r'([.,;:!?])(?!\s)', r'\1 ', text)
    # domain
    #text = re.sub(r'\.(?!\s|[a-zA-Z])', r'. ', text)
    # hapus spasi berlebih setelah penambahan
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [190]:
df['konten_berita'] = df['konten_berita'].apply(clean_text)
print(df[['konten_berita']].head())

                                       konten_berita
0  Tepat pada satu tahun pemerintahannya, Preside...
1  Sebuah kisah tentang anak-anak yang sekolahnya...
2  Di balik kenyamanan memiliki tempat tinggal se...
3  Di tengah lautan map cokelat dan antrean yang ...
4  Seiring dengan peningkatan pemahaman dan kebut...


In [191]:
df['judul_berita'] = df['judul_berita'].apply(clean_text)
print(df[['judul_berita']].head())

                                        judul_berita
0  Pidato Lengkap Presiden Prabowo dalam Sidang K...
1  Mimpi yang Tak Padam di Tapal Batas: Cerita An...
2  Antara Asa dan Cicilan: Kisah dan Strategi Par...
3  Yang Muda, yang Tua, yang Terlupa: Realitas Pa...
4  Mengupas Masa Depan Nissan di Indonesia, mulai...


## Hapus Atribut Teks Tidak Penting

In [192]:
def clean_kompas_text(text: str) -> str:
    # ubah kompas. com menjadi kompas.com
    text = re.sub(r'(?i)kompas\s*\.\s*com', 'kompas.com', text)

    # hapus prefiks lokasi, KOMPAS.com – ...
    text = re.sub(
        r'^[A-Z\s]+,\s*kompas\.com\s*[–\-—]\s*',
        '',
        text,
        flags=re.IGNORECASE
    )

    # hapus kompas doc
    text = re.sub(r'(?is)kompas\.com/\s*dok\.[^.]*\.', '', text)

    # hapus kompas.com
    text = re.sub(r'(?i)(?:[A-Za-z]*\s*[-–\\\/]*)?kompas\.com[\\/A-Za-z0-9\-]*', '', text)

    # gabungan
    text = re.sub(r'^[A-Za-z\s]*?[a-z](?=[A-Z])', '', text)

    # hapus baca, baca juga, dan kombinasinya
    text = re.sub(
        r'(?i)\(?baca(?:\s+(?:juga|selengkapnya))?\s*[:：]\s*[^.!?\n]*(?=[.!?\n]|$)',
        '',
        text
    )

    # mention
    text = re.sub(r'\(@[^)]+\)', '', text)
    text = re.sub(r'@\w+', '', text)

    # kredit kompas.com
    text = re.sub(r'(?i)\bkompas\.com/[\w\s.]+?[a-z](?=\s*[A-Z]{2,}|[A-Z][a-z]*[A-Z])', '', text)

    # hapus .com yang mungkin tersisa
    text = re.sub(r'\b\w*\.com\b', '', text, flags=re.IGNORECASE)

    # tanda baca dan spasi
    text = re.sub(r'\s+([.,!?;:])', r'\1', text)
    text = re.sub(r'([.,!?;:])([^\s"”’])', r'\1 \2', text)
    text = re.sub(r'(?<=\w)\(', r' (', text)
    text = re.sub(r'\(\s+', '(', text)
    text = re.sub(r'\s+\)', ')', text)
    text = re.sub(r'\)(?=[A-Za-z0-9])', r') ', text)
    text = re.sub(r'\s+(")', r'\1', text)
    text = re.sub(r'\.\s*"', '. "', text)

    # tanda baca berulang
    text = re.sub(r'([.,!?;:])(\s*\1)+', r'\1', text)
    text = re.sub(r'([.,!?;:])\s*[.,!?;:]+', r'\1', text)
    text = re.sub(r'"\s*"', '"', text)
    text = re.sub(r'""+', '"', text)

    # non huruf awal karakter
    text = re.sub(r'^[^A-Za-z0-9]+', '', text)

    # spasi ganda
    text = re.sub(r'\s{2,}', ' ', text)

    # trim spasi
    text = text.strip()

    return text

In [194]:
df['konten_berita'] = df['konten_berita'].apply(clean_kompas_text)
print(df[['konten_berita']].head())

                                       konten_berita
0  Tepat pada satu tahun pemerintahannya, Preside...
1  Sebuah kisah tentang anak-anak yang sekolahnya...
2  Di balik kenyamanan memiliki tempat tinggal se...
3  Di tengah lautan map cokelat dan antrean yang ...
4  Seiring dengan peningkatan pemahaman dan kebut...


In [195]:
df['judul_berita'] = df['judul_berita'].apply(clean_kompas_text)
print(df[['judul_berita']].head())

                                        judul_berita
0  Pidato Lengkap Presiden Prabowo dalam Sidang K...
1  Mimpi yang Tak Padam di Tapal Batas: Cerita An...
2  Antara Asa dan Cicilan: Kisah dan Strategi Par...
3  Yang Muda, yang Tua, yang Terlupa: Realitas Pa...
4  Mengupas Masa Depan Nissan di Indonesia, mulai...


In [196]:
df["judul_berita"] = df["judul_berita"].apply(
    lambda x: x.strip() + "." if not re.search(r'[.!?…]$', str(x).strip()) else x.strip()
)

## Save Hasil Preprocessing

In [197]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 499 entries, 0 to 508
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   category         499 non-null    object
 1   subcategory      499 non-null    object
 2   judul_berita     499 non-null    object
 3   url              499 non-null    object
 4   summary          499 non-null    object
 5   tanggal_publish  499 non-null    object
 6   konten_berita    499 non-null    object
dtypes: object(7)
memory usage: 31.2+ KB


In [198]:
df.duplicated().sum()

0

In [200]:
df.to_csv("../data/preprocessing/prepro_jeo.csv", index=False)

# Kata Netizen

In [201]:
df = pd.read_csv("../data/raw/katanetizen.csv")
df.head()

,url,category,subcategory,tanggal_publish,judul,tags,konten
0,https://katanetizen.kompas.com/read/2025/10/31...,Katanetizen,Kata Netizen,"Kompas.com - 31/10/2025, 16:24 WIB",Mencecap Masa Lalu lewat Es Krim di Kedai Jadul,"Kompasiana, es krim, jadul, kedai, Kata Netizen",Ternyata bisa lho ketika kita mencicip sesuatu...
1,https://katanetizen.kompas.com/read/2025/10/31...,Katanetizen,Kata Netizen,"Kompas.com - 31/10/2025, 15:07 WIB","Kini CFD Cibinong Tanpa Penjual Jajanan, Ada y...","Kompasiana, Kabupaten Bogor, CFD, Kata Netizen...","Car free day (CFD) di Cibinong, Kabupaten Bogo..."
2,https://katanetizen.kompas.com/read/2025/10/31...,Katanetizen,Kata Netizen,"Kompas.com - 31/10/2025, 13:58 WIB","Jalan-jalan ke Pasar Buku Legendaris Kwitang, ...","Kompasiana, toko buku, buku, Kwitang, Kata Net...",Apa yang terngiang darimu ketika mendengar kat...
3,https://katanetizen.kompas.com/read/2025/10/31...,Katanetizen,Kata Netizen,"Kompas.com - 31/10/2025, 13:06 WIB",Dunia Global Mesti Waspada Ancaman Penyakit Fl...,"virus, Kompasiana, flu burung, unggas, Kata Ne...",Apakah dunia benar-benar siap menghadapi ancam...
4,https://katanetizen.kompas.com/read/2025/10/31...,Katanetizen,Kata Netizen,"Kompas.com - 31/10/2025, 09:18 WIB",Melihat Sekolah di Korea Selatan Mengurangi Sa...,"Korea Selatan, Kompasiana, sampah makanan, Kat...",Kompasianer Yulius Roma Patandean Paket makan ...


In [202]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   url              15 non-null     object
 1   category         15 non-null     object
 2   subcategory      15 non-null     object
 3   tanggal_publish  15 non-null     object
 4   judul            15 non-null     object
 5   tags             15 non-null     object
 6   konten           15 non-null     object
dtypes: object(7)
memory usage: 968.0+ bytes


In [203]:
df.rename(columns={
    'judul': 'judul_berita',
    'konten': 'konten_berita'
}, inplace=True)

In [204]:
df.columns.tolist()

['url',
 'category',
 'subcategory',
 'tanggal_publish',
 'judul_berita',
 'tags',
 'konten_berita']

## Null Handling

In [205]:
df['konten_berita'].isna().sum()

0

## Remove Duplicate

In [206]:
df.duplicated().sum()

0

## Perbaiki Penulisan

In [207]:
import re

def remove_before_first_number(text: str) -> str:
    text = re.sub(r'^[^\d]*', '', text).strip()
    text = re.sub(r':\s+', ':', text)
    
    return text


In [208]:
print(df[['tanggal_publish']].head())

                      tanggal_publish
0  Kompas.com - 31/10/2025, 16:24 WIB
1  Kompas.com - 31/10/2025, 15:07 WIB
2  Kompas.com - 31/10/2025, 13:58 WIB
3  Kompas.com - 31/10/2025, 13:06 WIB
4  Kompas.com - 31/10/2025, 09:18 WIB


In [209]:
df['tanggal_publish'] = df['tanggal_publish'].apply(remove_before_first_number)
print(df[['tanggal_publish']].head())

         tanggal_publish
0  31/10/2025, 16:24 WIB
1  31/10/2025, 15:07 WIB
2  31/10/2025, 13:58 WIB
3  31/10/2025, 13:06 WIB
4  31/10/2025, 09:18 WIB


In [210]:
def clean_text(text):
    # mengganti newline dengan spasi
    text = text.replace("\n", " ")
    # hapus spasi berlebih
    text = re.sub(r'\s+', ' ', text).strip()
    # spasi setelah tanda baca jika belum ada
    text = re.sub(r'([.,;:!?])(?!\s)', r'\1 ', text)
    # domain
    #text = re.sub(r'\.(?!\s|[a-zA-Z])', r'. ', text)
    # hapus spasi berlebih setelah penambahan
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [211]:
df['konten_berita'] = df['konten_berita'].apply(clean_text)
print(df[['konten_berita']].head())

                                       konten_berita
0  Ternyata bisa lho ketika kita mencicip sesuatu...
1  Car free day (CFD) di Cibinong, Kabupaten Bogo...
2  Apa yang terngiang darimu ketika mendengar kat...
3  Apakah dunia benar-benar siap menghadapi ancam...
4  Kompasianer Yulius Roma Patandean Paket makan ...


In [212]:
df['judul_berita'] = df['judul_berita'].apply(clean_text)
print(df[['judul_berita']].head())

                                        judul_berita
0    Mencecap Masa Lalu lewat Es Krim di Kedai Jadul
1  Kini CFD Cibinong Tanpa Penjual Jajanan, Ada y...
2  Jalan-jalan ke Pasar Buku Legendaris Kwitang, ...
3  Dunia Global Mesti Waspada Ancaman Penyakit Fl...
4  Melihat Sekolah di Korea Selatan Mengurangi Sa...


## Hapus Atribut Teks Tidak Penting

In [213]:
def clean_kompas_text(text: str) -> str:
    # ubah kompas. com menjadi kompas.com
    text = re.sub(r'(?i)kompas\s*\.\s*com', 'kompas.com', text)

    # hapus prefiks lokasi, KOMPAS.com – ...
    text = re.sub(
        r'^[A-Z\s]+,\s*kompas\.com\s*[–\-—]\s*',
        '',
        text,
        flags=re.IGNORECASE
    )

    # hapus kompas doc
    text = re.sub(r'(?is)kompas\.com/\s*dok\.[^.]*\.', '', text)

    # hapus kompas.com
    text = re.sub(r'(?i)(?:[A-Za-z]*\s*[-–\\\/]*)?kompas\.com[\\/A-Za-z0-9\-]*', '', text)

    # gabungan
    text = re.sub(r'^[A-Za-z\s]*?[a-z](?=[A-Z])', '', text)

    # hapus baca, baca juga, dan kombinasinya
    text = re.sub(
        r'(?i)\(?baca(?:\s+(?:juga|selengkapnya))?\s*[:：]\s*[^.!?\n]*(?=[.!?\n]|$)',
        '',
        text
    )

    # mention
    text = re.sub(r'\(@[^)]+\)', '', text)
    text = re.sub(r'@\w+', '', text)

    # kredit kompas.com
    text = re.sub(r'(?i)\bkompas\.com/[\w\s.]+?[a-z](?=\s*[A-Z]{2,}|[A-Z][a-z]*[A-Z])', '', text)

    # hapus .com yang mungkin tersisa
    text = re.sub(r'\b\w*\.com\b', '', text, flags=re.IGNORECASE)

    # tanda baca dan spasi
    text = re.sub(r'\s+([.,!?;:])', r'\1', text)
    text = re.sub(r'([.,!?;:])([^\s"”’])', r'\1 \2', text)
    text = re.sub(r'(?<=\w)\(', r' (', text)
    text = re.sub(r'\(\s+', '(', text)
    text = re.sub(r'\s+\)', ')', text)
    text = re.sub(r'\)(?=[A-Za-z0-9])', r') ', text)
    text = re.sub(r'\s+(")', r'\1', text)
    text = re.sub(r'\.\s*"', '. "', text)

    # tanda baca berulang
    text = re.sub(r'([.,!?;:])(\s*\1)+', r'\1', text)
    text = re.sub(r'([.,!?;:])\s*[.,!?;:]+', r'\1', text)
    text = re.sub(r'"\s*"', '"', text)
    text = re.sub(r'""+', '"', text)

    # non huruf awal karakter
    text = re.sub(r'^[^A-Za-z0-9]+', '', text)

    # spasi ganda
    text = re.sub(r'\s{2,}', ' ', text)

    # trim spasi
    text = text.strip()

    return text

In [214]:
print(df[['konten_berita']].head())

                                       konten_berita
0  Ternyata bisa lho ketika kita mencicip sesuatu...
1  Car free day (CFD) di Cibinong, Kabupaten Bogo...
2  Apa yang terngiang darimu ketika mendengar kat...
3  Apakah dunia benar-benar siap menghadapi ancam...
4  Kompasianer Yulius Roma Patandean Paket makan ...


In [215]:
df['konten_berita'] = df['konten_berita'].apply(clean_kompas_text)
print(df[['konten_berita']].head())

                                       konten_berita
0  Ternyata bisa lho ketika kita mencicip sesuatu...
1  Car free day (CFD) di Cibinong, Kabupaten Bogo...
2  Apa yang terngiang darimu ketika mendengar kat...
3  Apakah dunia benar-benar siap menghadapi ancam...
4  Kompasianer Yulius Roma Patandean Paket makan ...


In [216]:
print(df[['judul_berita']].head())

                                        judul_berita
0    Mencecap Masa Lalu lewat Es Krim di Kedai Jadul
1  Kini CFD Cibinong Tanpa Penjual Jajanan, Ada y...
2  Jalan-jalan ke Pasar Buku Legendaris Kwitang, ...
3  Dunia Global Mesti Waspada Ancaman Penyakit Fl...
4  Melihat Sekolah di Korea Selatan Mengurangi Sa...


In [217]:
df['judul_berita'] = df['judul_berita'].apply(clean_kompas_text)
print(df[['judul_berita']].head())

                                        judul_berita
0    Mencecap Masa Lalu lewat Es Krim di Kedai Jadul
1  Kini CFD Cibinong Tanpa Penjual Jajanan, Ada y...
2  Jalan-jalan ke Pasar Buku Legendaris Kwitang, ...
3  Dunia Global Mesti Waspada Ancaman Penyakit Fl...
4  Melihat Sekolah di Korea Selatan Mengurangi Sa...


In [218]:
df["judul_berita"] = df["judul_berita"].apply(
    lambda x: x.strip() + "." if not re.search(r'[.!?…]$', str(x).strip()) else x.strip()
)

## Save Hasil Preprocessing

In [219]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   url              15 non-null     object
 1   category         15 non-null     object
 2   subcategory      15 non-null     object
 3   tanggal_publish  15 non-null     object
 4   judul_berita     15 non-null     object
 5   tags             15 non-null     object
 6   konten_berita    15 non-null     object
dtypes: object(7)
memory usage: 968.0+ bytes


In [220]:
df.duplicated().sum()

0

In [221]:
df.to_csv("../data/preprocessing/prepro_katanetizen.csv", index=False)

# Kolom

In [222]:
df = pd.read_csv("../data/raw/kolom.csv")
df.head()

,url,category,subcategory,tanggal_publish,judul,tags,konten
0,http://www.kompas.com/edu/read/2025/11/12/1433...,Edu,Edu,"12 November 2025, 14:33 WIB",Masa Depan Anak Berhadapan dengan Hukum,"bullying, perundungan, ledakan di sma 72 jakarta","KASUS ledakan di SMAN 72 Jakarta, Kelapa Gadin..."
1,http://agri.kompas.com/read/2025/11/12/1200009...,Agri,Varietas Tanaman,"Kompas.com - 12 November 2025, 12:00 WIB",Masa Depan Pala Banda,"ekspor pala, pala banda, produk perkebunan Mal...","BEBERAPA abad silam, Kepulauan Banda di Maluku..."
2,http://nasional.kompas.com/read/2025/11/12/104...,News,Nasional,"12 November 2025, 10:48 WIB",Ketika Marsinah dan Soeharto di Bingkai yang Sama,"Soeharto, Marsinah, pahlawan nasional",HARI Pahlawan tahun ini menghadirkan pemandang...
3,http://regional.kompas.com/read/2025/11/12/093...,News,Regional,"12 November 2025, 09:30 WIB",Tuan Rondahaim dan Semangatnya Kini,"Tuan Rondahaim Saragih Garingging, PT Toba Pul...","SENIN , 10 November 2025, merupakan hari penga..."
4,http://www.kompas.com/edu/read/2025/11/12/0845...,Edu,Edu,"12 November 2025, 08:45 WIB","Neng-Ning-Nung (3): Bapak dan Ibu Guru, Jangan...","guru, pahlawan tanpa tanda jasa",Neng: keheningan batin (meneng); Ning: hati lu...


In [223]:
df.rename(columns={
    'judul': 'judul_berita',
    'konten': 'konten_berita'
}, inplace=True)

In [224]:
df.columns.tolist()

['url',
 'category',
 'subcategory',
 'tanggal_publish',
 'judul_berita',
 'tags',
 'konten_berita']

## Null Handling

In [225]:
df['konten_berita'].isna().sum()

0

## Remove Duplicate

In [226]:
df.duplicated().sum()

0

## Perbaiki Penulisan

In [227]:
def clean_text(text):
    # mengganti newline dengan spasi
    text = text.replace("\n", " ")
    # hapus spasi berlebih
    text = re.sub(r'\s+', ' ', text).strip()
    # spasi setelah tanda baca jika belum ada
    text = re.sub(r'([.,;:!?])(?!\s)', r'\1 ', text)
    # domain
    #text = re.sub(r'\.(?!\s|[a-zA-Z])', r'. ', text)
    # hapus spasi berlebih setelah penambahan
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [228]:
df['konten_berita'] = df['konten_berita'].apply(clean_text)
print(df[['konten_berita']].head())

                                       konten_berita
0  KASUS ledakan di SMAN 72 Jakarta, Kelapa Gadin...
1  BEBERAPA abad silam, Kepulauan Banda di Maluku...
2  HARI Pahlawan tahun ini menghadirkan pemandang...
3  SENIN , 10 November 2025, merupakan hari penga...
4  Neng: keheningan batin (meneng); Ning: hati lu...


In [229]:
df['judul_berita'] = df['judul_berita'].apply(clean_text)
print(df[['judul_berita']].head())

                                        judul_berita
0            Masa Depan Anak Berhadapan dengan Hukum
1                              Masa Depan Pala Banda
2  Ketika Marsinah dan Soeharto di Bingkai yang Sama
3                Tuan Rondahaim dan Semangatnya Kini
4  Neng-Ning-Nung (3): Bapak dan Ibu Guru, Jangan...


## Hapus Atribut Teks Tidak Penting

In [230]:
import re

def clean_kompas_text(text: str) -> str:
    # ubah kompas. com menjadi kompas.com
    text = re.sub(r'(?i)kompas\s*\.\s*com', 'kompas.com', text)

    # hapus prefiks lokasi, KOMPAS.com – ...
    text = re.sub(
        r'^[A-Z\s]+,\s*kompas\.com\s*[–\-—]\s*',
        '',
        text,
        flags=re.IGNORECASE
    )

    # hapus kompas doc
    text = re.sub(r'(?is)kompas\.com/\s*dok\.[^.]*\.', '', text)

    # hapus kompas.com
    text = re.sub(r'(?i)(?:[A-Za-z]*\s*[-–\\\/]*)?kompas\.com[\\/A-Za-z0-9\-]*', '', text)

    # gabungan
    text = re.sub(r'^[A-Za-z\s]*?[a-z](?=[A-Z])', '', text)

    # hapus baca, baca juga, dan kombinasinya
    text = re.sub(
        r'(?i)\(?baca(?:\s+(?:juga|selengkapnya))?\s*[:：]\s*[^.!?\n]*(?=[.!?\n]|$)',
        '',
        text
    )

    # mention
    text = re.sub(r'\(@[^)]+\)', '', text)
    text = re.sub(r'@\w+', '', text)

    # kredit kompas.com
    text = re.sub(r'(?i)\bkompas\.com/[\w\s.]+?[a-z](?=\s*[A-Z]{2,}|[A-Z][a-z]*[A-Z])', '', text)

    # hapus .com yang mungkin tersisa
    text = re.sub(r'\b\w*\.com\b', '', text, flags=re.IGNORECASE)

    # tanda baca dan spasi
    text = re.sub(r'\s+([.,!?;:])', r'\1', text)
    text = re.sub(r'([.,!?;:])([^\s"”’])', r'\1 \2', text)
    text = re.sub(r'(?<=\w)\(', r' (', text)
    text = re.sub(r'\(\s+', '(', text)
    text = re.sub(r'\s+\)', ')', text)
    text = re.sub(r'\)(?=[A-Za-z0-9])', r') ', text)
    text = re.sub(r'\s+(")', r'\1', text)
    text = re.sub(r'\.\s*"', '. "', text)

    # tanda baca berulang
    text = re.sub(r'([.,!?;:])(\s*\1)+', r'\1', text)
    text = re.sub(r'([.,!?;:])\s*[.,!?;:]+', r'\1', text)
    text = re.sub(r'"\s*"', '"', text)
    text = re.sub(r'""+', '"', text)

    # non huruf awal karakter
    text = re.sub(r'^[^A-Za-z0-9]+', '', text)

    # spasi ganda
    text = re.sub(r'\s{2,}', ' ', text)

    # trim spasi
    text = text.strip()

    return text


In [231]:
print(df[['konten_berita']].head())

                                       konten_berita
0  KASUS ledakan di SMAN 72 Jakarta, Kelapa Gadin...
1  BEBERAPA abad silam, Kepulauan Banda di Maluku...
2  HARI Pahlawan tahun ini menghadirkan pemandang...
3  SENIN , 10 November 2025, merupakan hari penga...
4  Neng: keheningan batin (meneng); Ning: hati lu...


In [232]:
df['konten_berita'] = df['konten_berita'].apply(clean_kompas_text)
print(df[['konten_berita']].head())

                                       konten_berita
0  KASUS ledakan di SMAN 72 Jakarta, Kelapa Gadin...
1  BEBERAPA abad silam, Kepulauan Banda di Maluku...
2  HARI Pahlawan tahun ini menghadirkan pemandang...
3  SENIN, 10 November 2025, merupakan hari pengan...
4  Neng: keheningan batin (meneng); Ning: hati lu...


In [233]:
df['judul_berita'] = df['judul_berita'].apply(clean_kompas_text)
print(df[['judul_berita']].head())

                                        judul_berita
0            Masa Depan Anak Berhadapan dengan Hukum
1                              Masa Depan Pala Banda
2  Ketika Marsinah dan Soeharto di Bingkai yang Sama
3                Tuan Rondahaim dan Semangatnya Kini
4  Neng-Ning-Nung (3): Bapak dan Ibu Guru, Jangan...


In [234]:
df["judul_berita"] = df["judul_berita"].apply(
    lambda x: x.strip() + "." if not re.search(r'[.!?…]$', str(x).strip()) else x.strip()
)

## Save Hasil Preprocessing

In [235]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1593 entries, 0 to 1592
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   url              1593 non-null   object
 1   category         1593 non-null   object
 2   subcategory      1593 non-null   object
 3   tanggal_publish  1593 non-null   object
 4   judul_berita     1593 non-null   object
 5   tags             1593 non-null   object
 6   konten_berita    1593 non-null   object
dtypes: object(7)
memory usage: 87.2+ KB


In [236]:
df.duplicated().sum()

0

In [237]:
df.to_csv("../data/preprocessing/prepro_kolom.csv", index=False)

## Tren

In [238]:
df = pd.read_csv("../data/raw/tren.csv")
df.head()

,url,category,subcategory,tanggal_publish,judul,tags,konten
0,https://www.kompas.com/tren/read/2025/11/01/21...,Kompas,Tren,"1 November 2025, 21:00 WIB",Wanita Gagal Ginjal dan Pria Kanker Menikah ka...,"China, pasien kanker, transplantasi ginjal, ki...",KOMPAS.com - Kisah cinta tak terduga datang da...
1,https://www.kompas.com/tren/read/2025/11/01/20...,Kompas,Tren,"1 November 2025, 20:30 WIB","10 Negara Bebas Pajak Penghasilan pada 2025, N...","Qatar, Pajak penghasilan, Bahrain, negara yang...",KOMPAS.com - Pajak merupakan salah satu elemen...
2,https://www.kompas.com/tren/read/2025/11/01/20...,Kompas,Tren,"1 November 2025, 20:15 WIB","Banjir Besar Melanda New York, Hujan Pecahkan ...","New York, hujan lebat, New York banjir, banjir...",KOMPAS.com - Hujan ekstrem yang melanda New Yo...
3,https://www.kompas.com/tren/read/2025/11/01/20...,Kompas,Tren,"1 November 2025, 20:00 WIB","Muncul Awan Berwarna-warni di Atas Yogyakarta,...","BMKG, Cloud iridescence, fire rainbow, fenomen...",KOMPAS.com - Unggahan foto yang memperlihatkan...
4,https://www.kompas.com/tren/read/2025/11/01/19...,Kompas,Tren,"1 November 2025, 19:30 WIB",Ramai Video Suara Guntur Terdengar Lebih dari ...,"BMKG, yogyakarta, gemuruh dan petir, penyebab ...",KOMPAS.com - Sebuah video menampakkan langit g...


In [239]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542 entries, 0 to 541
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   url              542 non-null    object
 1   category         542 non-null    object
 2   subcategory      542 non-null    object
 3   tanggal_publish  542 non-null    object
 4   judul            542 non-null    object
 5   tags             542 non-null    object
 6   konten           542 non-null    object
dtypes: object(7)
memory usage: 29.8+ KB


In [240]:
df.rename(columns={
    'judul': 'judul_berita',
    'konten': 'konten_berita'
}, inplace=True)

In [241]:
df.columns.tolist()

['url',
 'category',
 'subcategory',
 'tanggal_publish',
 'judul_berita',
 'tags',
 'konten_berita']

## Null Handling

In [242]:
df['konten_berita'].isna().sum()

0

## Remove Duplicate

In [243]:
df.duplicated().sum()

0

## Perbaiki Penulisan

In [244]:
def clean_text(text):
    # mengganti newline dengan spasi
    text = text.replace("\n", " ")
    # hapus spasi berlebih
    text = re.sub(r'\s+', ' ', text).strip()
    # spasi setelah tanda baca jika belum ada
    text = re.sub(r'([.,;:!?])(?!\s)', r'\1 ', text)
    # domain
    #text = re.sub(r'\.(?!\s|[a-zA-Z])', r'. ', text)
    # hapus spasi berlebih setelah penambahan
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [245]:
df['konten_berita'] = df['konten_berita'].apply(clean_text)
print(df[['konten_berita']].head())

                                       konten_berita
0  KOMPAS. com - Kisah cinta tak terduga datang d...
1  KOMPAS. com - Pajak merupakan salah satu eleme...
2  KOMPAS. com - Hujan ekstrem yang melanda New Y...
3  KOMPAS. com - Unggahan foto yang memperlihatka...
4  KOMPAS. com - Sebuah video menampakkan langit ...


In [246]:
df['judul_berita'] = df['judul_berita'].apply(clean_text)
print(df[['judul_berita']].head())

                                        judul_berita
0  Wanita Gagal Ginjal dan Pria Kanker Menikah ka...
1  10 Negara Bebas Pajak Penghasilan pada 2025, N...
2  Banjir Besar Melanda New York, Hujan Pecahkan ...
3  Muncul Awan Berwarna-warni di Atas Yogyakarta,...
4  Ramai Video Suara Guntur Terdengar Lebih dari ...


## Hapus Atribut Teks Tidak Penting

In [247]:
import re

def clean_kompas_text(text: str) -> str:
    # ubah kompas. com menjadi kompas.com
    text = re.sub(r'(?i)kompas\s*\.\s*com', 'kompas.com', text)

    # hapus prefiks lokasi, KOMPAS.com – ...
    text = re.sub(
        r'^[A-Z\s]+,\s*kompas\.com\s*[–\-—]\s*',
        '',
        text,
        flags=re.IGNORECASE
    )

    # hapus kompas doc
    text = re.sub(r'(?is)kompas\.com/\s*dok\.[^.]*\.', '', text)

    # hapus kompas.com
    text = re.sub(r'(?i)(?:[A-Za-z]*\s*[-–\\\/]*)?kompas\.com[\\/A-Za-z0-9\-]*', '', text)

    # gabungan
    text = re.sub(r'^[A-Za-z\s]*?[a-z](?=[A-Z])', '', text)

    # hapus baca, baca juga, dan kombinasinya
    text = re.sub(
        r'(?i)\(?baca(?:\s+(?:juga|selengkapnya))?\s*[:：]\s*[^.!?\n]*(?=[.!?\n]|$)',
        '',
        text
    )

    # mention
    text = re.sub(r'\(@[^)]+\)', '', text)
    text = re.sub(r'@\w+', '', text)

    # kredit kompas.com
    text = re.sub(r'(?i)\bkompas\.com/[\w\s.]+?[a-z](?=\s*[A-Z]{2,}|[A-Z][a-z]*[A-Z])', '', text)

    # hapus .com yang mungkin tersisa
    text = re.sub(r'\b\w*\.com\b', '', text, flags=re.IGNORECASE)

    # tanda baca dan spasi
    text = re.sub(r'\s+([.,!?;:])', r'\1', text)
    text = re.sub(r'([.,!?;:])([^\s"”’])', r'\1 \2', text)
    text = re.sub(r'(?<=\w)\(', r' (', text)
    text = re.sub(r'\(\s+', '(', text)
    text = re.sub(r'\s+\)', ')', text)
    text = re.sub(r'\)(?=[A-Za-z0-9])', r') ', text)
    text = re.sub(r'\s+(")', r'\1', text)
    text = re.sub(r'\.\s*"', '. "', text)

    # tanda baca berulang
    text = re.sub(r'([.,!?;:])(\s*\1)+', r'\1', text)
    text = re.sub(r'([.,!?;:])\s*[.,!?;:]+', r'\1', text)
    text = re.sub(r'"\s*"', '"', text)
    text = re.sub(r'""+', '"', text)

    # non huruf awal karakter
    text = re.sub(r'^[^A-Za-z0-9]+', '', text)

    # spasi ganda
    text = re.sub(r'\s{2,}', ' ', text)

    # trim spasi
    text = text.strip()

    return text


In [248]:
print(df[['konten_berita']].head())

                                       konten_berita
0  KOMPAS. com - Kisah cinta tak terduga datang d...
1  KOMPAS. com - Pajak merupakan salah satu eleme...
2  KOMPAS. com - Hujan ekstrem yang melanda New Y...
3  KOMPAS. com - Unggahan foto yang memperlihatka...
4  KOMPAS. com - Sebuah video menampakkan langit ...


In [249]:
df['konten_berita'] = df['konten_berita'].apply(clean_kompas_text)
print(df[['konten_berita']].head())

                                       konten_berita
0  Kisah cinta tak terduga datang dari Provinsi S...
1  Pajak merupakan salah satu elemen penting dala...
2  Hujan ekstrem yang melanda New York City pada ...
3  Unggahan foto yang memperlihatkan cahaya berwa...
4  Sebuah video menampakkan langit gelap diselimu...


In [250]:
df['judul_berita'] = df['judul_berita'].apply(clean_kompas_text)
print(df[['judul_berita']].head())

                                        judul_berita
0  Wanita Gagal Ginjal dan Pria Kanker Menikah ka...
1  10 Negara Bebas Pajak Penghasilan pada 2025, N...
2  Banjir Besar Melanda New York, Hujan Pecahkan ...
3  Muncul Awan Berwarna-warni di Atas Yogyakarta,...
4  Ramai Video Suara Guntur Terdengar Lebih dari ...


In [251]:
df["judul_berita"] = df["judul_berita"].apply(
    lambda x: x.strip() + "." if not re.search(r'[.!?…]$', str(x).strip()) else x.strip()
)

## Save Hasil Preprocessing

In [252]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542 entries, 0 to 541
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   url              542 non-null    object
 1   category         542 non-null    object
 2   subcategory      542 non-null    object
 3   tanggal_publish  542 non-null    object
 4   judul_berita     542 non-null    object
 5   tags             542 non-null    object
 6   konten_berita    542 non-null    object
dtypes: object(7)
memory usage: 29.8+ KB


In [253]:
df.duplicated().sum()

0

In [254]:
df.to_csv("../data/preprocessing/prepro_tren.csv", index=False)

# Warta

In [255]:
df = pd.read_csv("../data/raw/warta.csv")
df.head()

,url,category,subcategory,tanggal_publish,judul,tags,konten
0,http://warta.kompas.com/read/2025/11/09/090000...,Warta,Sukacita,"Kompas.com - 09/11/2025, 09:00 WIB","Merajut Harmoni Membangun Kebersamaan, Walkot ...",NaN,"KOMPAS.com - Pada 1510-1546, Raja Gowa ke-9, T..."
1,http://warta.kompas.com/read/2025/10/17/213506...,Warta,Sukacita,"Kompas.com - 17/10/2025, 21:35 WIB",Profil Presiden Prabowo yang Hari Ini Genap Be...,NaN,"KOMPAS.com - Hari ini, Jumat, 17 Oktober 2025,..."
2,http://warta.kompas.com/read/2025/10/04/090000...,Warta,Sukacita,"Kompas.com - 04/10/2025, 09:00 WIB","HUT Ke-25 Banten, Semangat ""Kolaborasi Kuat"" J...",NaN,"KOMPAS.com – Pada 4 Oktober 2000, ribuan masya..."
3,http://warta.kompas.com/read/2025/10/03/133000...,Warta,Sukacita,"Kompas.com - 03/10/2025, 13:30 WIB","Merayakan Keberlanjutan, Lestari Awards 2025 A...",NaN,"JAKARTA, KOMPAS.com – Malam penuh sukacita men..."
4,http://warta.kompas.com/read/2025/09/14/090000...,Warta,Sukacita,"Kompas.com - 14/09/2025, 09:00 WIB","Tiga Dekade Kompas.com, Menyalakan Semangat Li...",NaN,KOMPAS.com - Tiga dekade adalah usia yang mata...


In [256]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   url              5 non-null      object 
 1   category         5 non-null      object 
 2   subcategory      5 non-null      object 
 3   tanggal_publish  5 non-null      object 
 4   judul            5 non-null      object 
 5   tags             0 non-null      float64
 6   konten           5 non-null      object 
dtypes: float64(1), object(6)
memory usage: 408.0+ bytes


In [257]:
df.rename(columns={
    'judul': 'judul_berita',
    'konten': 'konten_berita'
}, inplace=True)

In [258]:
df.columns.tolist()

['url',
 'category',
 'subcategory',
 'tanggal_publish',
 'judul_berita',
 'tags',
 'konten_berita']

## Null Handling

In [259]:
df['konten_berita'].isna().sum()

0

## Remove Duplicate

In [260]:
df.duplicated().sum()

0

## Hapus Atribut Tags

In [261]:
df = df.drop('tags', axis=1)

In [262]:
df.columns.tolist()

['url',
 'category',
 'subcategory',
 'tanggal_publish',
 'judul_berita',
 'konten_berita']

## Perbaiki Penulisan

In [263]:
import re

def remove_before_first_number(text: str) -> str:
    text = re.sub(r'^[^\d]*', '', text).strip()
    text = re.sub(r':\s+', ':', text)
    
    return text


In [264]:
print(df[['tanggal_publish']].head())

                      tanggal_publish
0  Kompas.com - 09/11/2025, 09:00 WIB
1  Kompas.com - 17/10/2025, 21:35 WIB
2  Kompas.com - 04/10/2025, 09:00 WIB
3  Kompas.com - 03/10/2025, 13:30 WIB
4  Kompas.com - 14/09/2025, 09:00 WIB


In [265]:
df['tanggal_publish'] = df['tanggal_publish'].apply(remove_before_first_number)
print(df[['tanggal_publish']].head())

         tanggal_publish
0  09/11/2025, 09:00 WIB
1  17/10/2025, 21:35 WIB
2  04/10/2025, 09:00 WIB
3  03/10/2025, 13:30 WIB
4  14/09/2025, 09:00 WIB


In [266]:
def clean_text(text):
    # mengganti newline dengan spasi
    text = text.replace("\n", " ")
    # hapus spasi berlebih
    text = re.sub(r'\s+', ' ', text).strip()
    # spasi setelah tanda baca jika belum ada
    text = re.sub(r'([.,;:!?])(?!\s)', r'\1 ', text)
    # domain
    #text = re.sub(r'\.(?!\s|[a-zA-Z])', r'. ', text)
    # hapus spasi berlebih setelah penambahan
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [267]:
df['konten_berita'] = df['konten_berita'].apply(clean_text)
print(df[['konten_berita']].head())

                                       konten_berita
0  KOMPAS. com - Pada 1510-1546, Raja Gowa ke-9, ...
1  KOMPAS. com - Hari ini, Jumat, 17 Oktober 2025...
2  KOMPAS. com – Pada 4 Oktober 2000, ribuan masy...
3  JAKARTA, KOMPAS. com – Malam penuh sukacita me...
4  KOMPAS. com - Tiga dekade adalah usia yang mat...


In [268]:
df['judul_berita'] = df['judul_berita'].apply(clean_text)
print(df[['judul_berita']].head())

                                        judul_berita
0  Merajut Harmoni Membangun Kebersamaan, Walkot ...
1  Profil Presiden Prabowo yang Hari Ini Genap Be...
2  HUT Ke-25 Banten, Semangat "Kolaborasi Kuat" J...
3  Merayakan Keberlanjutan, Lestari Awards 2025 A...
4  Tiga Dekade Kompas. com, Menyalakan Semangat L...


## Hapus Atribut Teks Tidak Penting

In [269]:
def clean_kompas_text(text: str) -> str:
    # ubah kompas. com menjadi kompas.com
    text = re.sub(r'(?i)kompas\s*\.\s*com', 'kompas.com', text)

    # hapus prefiks lokasi, KOMPAS.com – ...
    text = re.sub(
        r'^[A-Z\s]+,\s*kompas\.com\s*[–\-—]\s*',
        '',
        text,
        flags=re.IGNORECASE
    )

    # hapus kompas doc
    text = re.sub(r'(?is)kompas\.com/\s*dok\.[^.]*\.', '', text)

    # hapus kompas.com
    text = re.sub(r'(?i)(?:[A-Za-z]*\s*[-–\\\/]*)?kompas\.com[\\/A-Za-z0-9\-]*', '', text)

    # gabungan
    text = re.sub(r'^[A-Za-z\s]*?[a-z](?=[A-Z])', '', text)

    # hapus baca, baca juga, dan kombinasinya
    text = re.sub(
        r'(?i)\(?baca(?:\s+(?:juga|selengkapnya))?\s*[:：]\s*[^.!?\n]*(?=[.!?\n]|$)',
        '',
        text
    )

    # mention
    text = re.sub(r'\(@[^)]+\)', '', text)
    text = re.sub(r'@\w+', '', text)

    # kredit kompas.com
    text = re.sub(r'(?i)\bkompas\.com/[\w\s.]+?[a-z](?=\s*[A-Z]{2,}|[A-Z][a-z]*[A-Z])', '', text)

    # hapus .com yang mungkin tersisa
    text = re.sub(r'\b\w*\.com\b', '', text, flags=re.IGNORECASE)

    # tanda baca dan spasi
    text = re.sub(r'\s+([.,!?;:])', r'\1', text)
    text = re.sub(r'([.,!?;:])([^\s"”’])', r'\1 \2', text)
    text = re.sub(r'(?<=\w)\(', r' (', text)
    text = re.sub(r'\(\s+', '(', text)
    text = re.sub(r'\s+\)', ')', text)
    text = re.sub(r'\)(?=[A-Za-z0-9])', r') ', text)
    text = re.sub(r'\s+(")', r'\1', text)
    text = re.sub(r'\.\s*"', '. "', text)

    # tanda baca berulang
    text = re.sub(r'([.,!?;:])(\s*\1)+', r'\1', text)
    text = re.sub(r'([.,!?;:])\s*[.,!?;:]+', r'\1', text)
    text = re.sub(r'"\s*"', '"', text)
    text = re.sub(r'""+', '"', text)

    # non huruf awal karakter
    text = re.sub(r'^[^A-Za-z0-9]+', '', text)

    # spasi ganda
    text = re.sub(r'\s{2,}', ' ', text)

    # trim spasi
    text = text.strip()

    return text


In [270]:
print(df[['konten_berita']].head())

                                       konten_berita
0  KOMPAS. com - Pada 1510-1546, Raja Gowa ke-9, ...
1  KOMPAS. com - Hari ini, Jumat, 17 Oktober 2025...
2  KOMPAS. com – Pada 4 Oktober 2000, ribuan masy...
3  JAKARTA, KOMPAS. com – Malam penuh sukacita me...
4  KOMPAS. com - Tiga dekade adalah usia yang mat...


In [271]:
df['konten_berita'] = df['konten_berita'].apply(clean_kompas_text)
print(df[['konten_berita']].head())

                                       konten_berita
0  Pada 1510-1546, Raja Gowa ke-9, Tumaparisi Kal...
1  Hari ini, Jumat, 17 Oktober 2025, Presiden Rep...
2  Pada 4 Oktober 2000, ribuan masyarakat Banten ...
3  Malam penuh sukacita menyelimuti Raffles Hotel...
4  Tiga dekade adalah usia yang matang bagi sebua...


In [272]:
print(df[['judul_berita']].head())

                                        judul_berita
0  Merajut Harmoni Membangun Kebersamaan, Walkot ...
1  Profil Presiden Prabowo yang Hari Ini Genap Be...
2  HUT Ke-25 Banten, Semangat "Kolaborasi Kuat" J...
3  Merayakan Keberlanjutan, Lestari Awards 2025 A...
4  Tiga Dekade Kompas. com, Menyalakan Semangat L...


In [273]:
df['judul_berita'] = df['judul_berita'].apply(clean_kompas_text)
print(df[['judul_berita']].head())

                                        judul_berita
0  Merajut Harmoni Membangun Kebersamaan, Walkot ...
1  Profil Presiden Prabowo yang Hari Ini Genap Be...
2  HUT Ke-25 Banten, Semangat"Kolaborasi Kuat" Ja...
3  Merayakan Keberlanjutan, Lestari Awards 2025 A...
4  Tiga, Menyalakan Semangat Literasi di Era Disr...


In [274]:
df["judul_berita"] = df["judul_berita"].apply(
    lambda x: x.strip() + "." if not re.search(r'[.!?…]$', str(x).strip()) else x.strip()
)

## Save Hasil Preprocessing

In [275]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   url              5 non-null      object
 1   category         5 non-null      object
 2   subcategory      5 non-null      object
 3   tanggal_publish  5 non-null      object
 4   judul_berita     5 non-null      object
 5   konten_berita    5 non-null      object
dtypes: object(6)
memory usage: 368.0+ bytes


In [276]:
df.duplicated().sum()

0

In [277]:
df.to_csv("../data/preprocessing/prepro_warta.csv", index=False)